# Create a local information system of a Thin-Section

### TEST files

requirements:

- Python 3
- GDAL

## 1. Input and settings

In this section we collect input data and global settings.
We assume that we have a raster file of a Thin-section digitalized in any way and we have defined a polygon feature that overlay minerals or other elements of the thin-section.

The poligon feature layer must be in a GIS-readable format.
We use gadl2tiles with no-crs definition

In [1]:
# verify Python version
import sys
if sys.version_info[0] < 3:
    raise Exception("Python 3 not detected.")
else :
    print("Python 3 detected. OK.")

# verify GDAL version
try:
    from osgeo import gdal
    print(f"GDAL version: {gdal.__version__}")
except ImportError:
    raise ImportError("GDAL not detected.")

Python 3 detected. OK.
GDAL version: 3.8.5


# 1. Import files

In this section we use `upload_files_widgets` from `lis_function.py` to import data and files usinng widgets.

GDAL toolbox do not support non-georefernced files, We assume as example that Thin Section is a maps in Aitoff projection (1 pixel is equivalent to a 1 meter of a canonical web map). But if only for tile generation and next for openlayer library visualization. the World_Aitoff reference system is classified by EPSG with the code: `54043`

In [3]:
# Input widgets

from lis_functions import upload_files_widgets
Shapefile_selector, Jpgfile_selector, projectname = upload_files_widgets()
# Shapefile_selector, Jpgfile_selector, projectname, EPSGcode = upload_files_widgets()


Label(value='Select ALL the shapefile files [.shp, .dbf, .shx, .cpg]')

FileUpload(value=(), accept='.shp, .dbf, .shx, .cpg', description='Shapefile polygon layer', multiple=True)

Label(value='Select the Image file')

FileUpload(value=(), description='Image')

Label(value='Select the project name')

Text(value='', description='Name:', placeholder='NAME')

Label(value='Note: ouptut folder will be created in the same directory as the input files')

In [4]:
from lis_functions import save_to_temp_dir

temp_raster_path, temp_shp_paths = save_to_temp_dir(Shapefile_selector, Jpgfile_selector)

# print(temp_raster_path)
# print(temp_shp_paths)

# Import subprocess module
import subprocess

# Raster file path from which to generate tiles
input_raster = temp_raster_path

# Directory di output per i tile
output_directory = 'temporary_files'

# #input CRS
# input_crs = 'EPSG:' + EPSGcode.placeholder


In [5]:

# Comando gdal2tiles.py
gdal2tiles_command = [
    'gdal2tiles.py',
    '-p', 'raster',
    '-z', '0-5',
    # '-s', input_crs, #IT IS WORK widhout CSR definition
    input_raster,
    output_directory
]
# Run GDAL subprocess
try:
    subprocess.run(gdal2tiles_command, check=True)
    print(f'Tile generati con successo nella directory: {output_directory}')
except subprocess.CalledProcessError as e:
    print(f'Errore durante la generazione dei tile: {e}')

Generating Base Tiles:


0...10...20...30...40...50...60...70...80...90...100 - done.
0...10

Generating Overview Tiles:


...20...30...40...50...60...70...80...90...100 - done.
Tile generati con successo nella directory: temporary_files


In [6]:
# zip the output directory and then remove it
import shutil
import os
zip_filename = f"{output_directory}.zip"
shutil.make_archive(output_directory, 'zip', output_directory)
print(f'compress: {zip_filename}')
shutil.rmtree(output_directory)



compress: temporary_files.zip


## TODO

- For further implementation read `tilemapsoruce.xml` file generated after gdal2tiles subprocess
- creating a customization of this and regenerate the `openlayers.html` file 